In [25]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
from keras.optimizers import Adam
from imageio import imread
import numpy as np
import matplotlib.patches as patches
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast

from data_generator.object_detection_2d_data_generator import DataGenerator
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.object_detection_2d_geometric_ops import Resize
from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

import tensorflow as tf
import keras as keras

# Load the trained model

In [26]:
def load_model(model_path, height, width):
    # 1: Build the Keras model
    K.clear_session() # Clear previous models from memory.

    if height == 300:
        model = ssd_300(image_size=(height, width, 3),
                        n_classes=20,
                        mode='inference',
                        l2_regularization=0.0005,
                        scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO are [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                        aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                                 [1.0, 2.0, 0.5],
                                                 [1.0, 2.0, 0.5]],
                        two_boxes_for_ar1=True,
                        steps=[8, 16, 32, 64, 100, 300],
                        offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                        clip_boxes=False,
                        variances=[0.1, 0.1, 0.2, 0.2],
                        normalize_coords=True,
                        subtract_mean=[123, 117, 104],
                        swap_channels=[2, 1, 0],
                        confidence_thresh=0.5,
                        iou_threshold=0.45,
                        top_k=200,
                        nms_max_output_size=400)
    elif height == 512:
        model = ssd_512(image_size=(height, width, 3),
                n_classes=20,
                mode='inference',
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
               steps=[8, 16, 32, 64, 128, 256, 512],
               offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               clip_boxes=False,
               variances=[0.1, 0.1, 0.2, 0.2],
               normalize_coords=True,
               subtract_mean=[123, 117, 104],
               swap_channels=[2, 1, 0],
               confidence_thresh=0.5,
               iou_threshold=0.45,
               top_k=200,
               nms_max_output_size=400)

    # 2: Load the trained weights into the model.
    # TODO: Set the path of the trained weights.
    model.load_weights(model_path, by_name=True)

    # 3: Compile the model so that Keras won't complain the next time you load it.

    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

    model.compile(optimizer=adam, loss=ssd_loss.compute_loss)
    
    return model

In [27]:
model_pascal_voc_07_plus_12_512x512 = '../weights/pascal_voc/pascal_voc_07_plus_12_512x512.h5'
model = load_model(model_pascal_voc_07_plus_12_512x512, 512, 512)

In [28]:
tf.train.write_graph(K.get_session().graph, "./extra_files", "ssd_model.pbtxt", as_text=True)
tf.train.write_graph(K.get_session().graph, "./extra_files", "ssd_model.pb", as_text=False)

'./extra_files/ssd_model.pb'

In [29]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph


In [54]:
graph = load_graph('./extra_files/converted_model.tflite')

/home/aldopedraza/anaconda3/envs/tensorflow2/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: Unexpected end-group tag: Not all data was converted


In [46]:
for op in graph.get_operations():
    print(op.name)

prefix/keras_learning_phase/input
prefix/keras_learning_phase
prefix/input_1
prefix/identity_layer/Identity
prefix/input_mean_normalization/sub/y
prefix/input_mean_normalization/sub
prefix/input_channel_swap/strided_slice/stack
prefix/input_channel_swap/strided_slice/stack_1
prefix/input_channel_swap/strided_slice/stack_2
prefix/input_channel_swap/strided_slice
prefix/input_channel_swap/strided_slice_1/stack
prefix/input_channel_swap/strided_slice_1/stack_1
prefix/input_channel_swap/strided_slice_1/stack_2
prefix/input_channel_swap/strided_slice_1
prefix/input_channel_swap/strided_slice_2/stack
prefix/input_channel_swap/strided_slice_2/stack_1
prefix/input_channel_swap/strided_slice_2/stack_2
prefix/input_channel_swap/strided_slice_2
prefix/input_channel_swap/stack
prefix/conv1_1/truncated_normal/shape
prefix/conv1_1/truncated_normal/mean
prefix/conv1_1/truncated_normal/stddev
prefix/conv1_1/truncated_normal/TruncatedNormal
prefix/conv1_1/truncated_normal/mul
prefix/conv1_1/truncated_n

prefix/IsVariableInitialized_24
prefix/IsVariableInitialized_25
prefix/IsVariableInitialized_26
prefix/IsVariableInitialized_27
prefix/IsVariableInitialized_28
prefix/IsVariableInitialized_29
prefix/IsVariableInitialized_30
prefix/IsVariableInitialized_31
prefix/IsVariableInitialized_32
prefix/IsVariableInitialized_33
prefix/IsVariableInitialized_34
prefix/IsVariableInitialized_35
prefix/IsVariableInitialized_36
prefix/IsVariableInitialized_37
prefix/IsVariableInitialized_38
prefix/IsVariableInitialized_39
prefix/IsVariableInitialized_40
prefix/IsVariableInitialized_41
prefix/IsVariableInitialized_42
prefix/IsVariableInitialized_43
prefix/IsVariableInitialized_44
prefix/IsVariableInitialized_45
prefix/IsVariableInitialized_46
prefix/IsVariableInitialized_47
prefix/IsVariableInitialized_48
prefix/IsVariableInitialized_49
prefix/IsVariableInitialized_50
prefix/IsVariableInitialized_51
prefix/IsVariableInitialized_52
prefix/IsVariableInitialized_53
prefix/IsVariableInitialized_54
prefix/I

In [45]:
saver = tf.train.Saver()
with tf.Session(graph=graph) as sess:
    tf.summary.FileWriter('./extra_files/logs', sess.graph)

In [14]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()
sess = keras.backend.get_session()
save_path = saver.save(sess, "./extra_files/ssd_model.ckpt")

## Freeze the Graph

In [37]:
# Freeze the graph
import tensorflow as tf
from tensorflow.python.tools import freeze_graph

input_graph_path = './extra_files/ssd_model.pb'
checkpoint_path = './extra_files/ssd_model.ckpt'
input_saver_def_path = ""
input_binary = True
output_node_names = "predictions/concat"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = './extra_files/ssd_frozen_model.pb'
clear_devices = True


freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from ./extra_files/ssd_model.ckpt
INFO:tensorflow:Froze 79 variables.
INFO:tensorflow:Converted 79 variables to const ops.


## Optimize for inference

In [38]:
!python3 -m tensorflow.python.tools.optimize_for_inference \
--input=./extra_files/ssd_frozen_model.pb \
--output=./extra_files/ssd_opt_model.pb \
--frozen_graph=True --input_names=input_1 \
--output_names=predictions/concat

In [ ]:
# summary model
bazel build tensorflow/tools/graph_transforms:summarize_graph && \
bazel-bin/tensorflow/tools/graph_transforms/summarize_graph \
--in_graph=../ssd/extra_files/ssd_frozen_model.pb

# Convert to tflite model

In [53]:
graph_def_file = "/home/aldopedraza/Documentos/ssd/extra_files/ssd_frozen_model.pb"
input_arrays = ["input_1"]
output_arrays = ["predictions/concat"]

converter = tf.contrib.lite.TocoConverter.from_frozen_graph(
  graph_def_file, input_arrays, output_arrays)
tflite_model = converter.convert()
open("./extra_files/converted_model.tflite", "wb").write(tflite_model)

109594732